In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Note: this cell can only be run once, since we're changing the directory.
import sys
import os

path = "D:/testings/Python/TestingPython/"
if path not in sys.path:
    sys.path.append(path)

os.chdir("../")
os.getcwd()

'D:\\testings\\Python\\TestingPython\\ModelFusion'

In [3]:
import torch
import pytorch_lightning as pl

from torch.utils.data import Subset
from ModelFusion.fusion.fuse_u_net import fuse_u_nets
from ModelFusion.helpers.load_data import load_data
from ModelFusion.helpers.load_model import reload_model
from ModelFusion.helpers.pl_helpers import TrainSeg, get_logger
from ModelFusion.helpers.pl_callbacks import ValVisualizationSeg
from pytorch_lightning.callbacks import ModelCheckpoint
from monai.utils import CommonKeys

In [4]:
args = {}
# input args
args["num_retrain_samples"] = 5
args["num_retrain_epochs"] = 1
args["save_dir"] = "./"
args["vendor"] = "A"

# fixed args
args["ds_name"] = "MNMS"
args["model_name"] = "UNet"
args["log_name"] = "u_net_eval_logs"
args["accelerator"] = "gpu" if torch.cuda.is_available() else "cpu"

## Fuse 2 UNets

In [31]:
# %%capture
model1_path = "./seg_logs/2022_11_22_02_40_15_864295/"
model2_path = "./seg_logs/2022_11_22_02_48_26_706789/"
model1 = reload_model(args["model_name"], model1_path)
model2 = reload_model(args["model_name"], model2_path)
model_fused = fuse_u_nets(model1_path, model2_path)

Previous layer shape is  None
Processing the coordinates to form ground_metric
dont leave off the squaring of the ground metric
dist: tensor([[1.0561, 1.0312, 0.9356, 0.9779],
        [0.9716, 1.0500, 1.0451, 1.0854],
        [1.1159, 0.9835, 0.8662, 0.9040],
        [1.0201, 0.9217, 1.0451, 0.9999]], device='cuda:0')
returns a uniform measure of cardinality:  4
returns a uniform measure of cardinality:  4
the transport map is  tensor([[0.0000e+00, 3.5968e-04, 1.3869e-01, 1.1096e-01],
        [2.5000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 3.8408e-05, 1.1118e-01, 1.3878e-01],
        [2.5615e-04, 2.4961e-01, 0.0000e+00, 1.2892e-04]], device='cuda:0')
Ratio of trace to the matrix sum:  tensor(0.1113, device='cuda:0')
Here, trace is 0.4452367126941681 and matrix sum is 3.9999985694885254 
col sum: tensor([1.0010, 1.0001, 0.9995, 0.9995], device='cuda:0'), row sum: tensor([1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0')
this is past correction for weight mode
Shap

returns a uniform measure of cardinality:  8
returns a uniform measure of cardinality:  8
the transport map is  tensor([[9.1615e-04, 1.9918e-03, 5.7190e-02, 7.5461e-03, 5.2212e-04, 3.6191e-02,
         1.2227e-02, 8.4157e-03],
        [3.1443e-02, 4.8591e-03, 7.8983e-04, 8.4496e-03, 8.0981e-03, 7.0470e-02,
         8.9013e-04, 6.1048e-07],
        [7.3380e-03, 3.8439e-02, 5.4238e-02, 2.9455e-03, 3.9697e-03, 1.6496e-02,
         1.5627e-03, 1.1200e-05],
        [2.5926e-02, 1.4734e-03, 4.4335e-03, 3.6541e-02, 3.7187e-02, 3.8650e-04,
         1.8912e-02, 1.3980e-04],
        [3.3851e-03, 6.6905e-04, 1.2006e-03, 1.4828e-02, 4.8800e-02, 4.3417e-04,
         4.9323e-02, 6.3605e-03],
        [5.0941e-02, 5.9472e-02, 5.0422e-04, 6.8637e-03, 3.6932e-03, 9.3772e-05,
         6.6530e-04, 2.7665e-03],
        [1.2245e-03, 8.1696e-03, 5.4380e-03, 4.4196e-02, 2.2333e-02, 2.7343e-04,
         4.0978e-02, 2.3872e-03],
        [3.8265e-03, 9.9263e-03, 1.2064e-03, 3.6293e-03, 3.9650e-04, 6.5438e-04,
  

model1 after: tensor([[-0.0157,  0.0216, -0.0176,  ..., -0.0625,  0.0006,  0.0343],
        [ 0.0075,  0.0051, -0.0131,  ...,  0.0028, -0.0219,  0.0039],
        [-0.0021,  0.0253,  0.0168,  ...,  0.0271, -0.0116,  0.0490],
        ...,
        [-0.0265, -0.0278, -0.0292,  ..., -0.0155, -0.0090, -0.0136],
        [-0.0331, -0.0780, -0.0620,  ..., -0.0214, -0.0513, -0.0888],
        [-0.0090, -0.0322, -0.0192,  ...,  0.0103, -0.0236, -0.0351]],
       device='cuda:0')
model2: tensor([[[-7.7470e-02,  6.4421e-02, -5.5020e-02,  ...,  5.6654e-02,
           1.0864e-02, -9.1199e-02],
         [-3.9945e-02, -1.6817e-02,  1.6481e-02,  ..., -7.5173e-02,
          -7.4342e-02,  4.1199e-02],
         [ 2.4023e-03, -3.3295e-02,  2.3358e-02,  ...,  1.2973e-01,
           1.2878e-01,  4.9258e-02],
         ...,
         [-8.7656e-02,  4.9056e-02, -5.0462e-02,  ..., -4.6081e-02,
          -6.8234e-02,  5.2755e-02],
         [-1.2762e-01, -1.0580e-01, -7.2955e-03,  ...,  6.0549e-02,
          -1.1671e

returns a uniform measure of cardinality:  32
returns a uniform measure of cardinality:  32
the transport map is  tensor([[1.9746e-04, 1.6707e-03, 5.7082e-04,  ..., 4.7614e-05, 9.1589e-04,
         7.2439e-04],
        [4.4695e-04, 1.9576e-03, 3.0497e-03,  ..., 9.2369e-04, 5.5221e-05,
         2.8858e-04],
        [1.9931e-04, 8.1498e-04, 3.2711e-03,  ..., 2.2067e-04, 3.8888e-04,
         1.2252e-03],
        ...,
        [3.5247e-04, 8.0911e-04, 7.5966e-04,  ..., 9.7513e-04, 2.7229e-03,
         1.0552e-03],
        [3.7052e-03, 4.7692e-04, 1.2507e-03,  ..., 4.8786e-04, 7.0125e-04,
         2.2269e-04],
        [2.7138e-04, 1.0378e-03, 8.1813e-04,  ..., 1.3595e-03, 1.8681e-03,
         4.9535e-04]], device='cuda:0')
Ratio of trace to the matrix sum:  tensor(0.0352, device='cuda:0')
Here, trace is 1.1253414154052734 and matrix sum is 31.999897003173828 
col sum: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.000

zeros
----------------------------------------------------------------------------------------------------
Previous layer shape is  None
Processing the coordinates to form ground_metric
dont leave off the squaring of the ground metric
dist: tensor([[0.9500, 0.9997, 0.9599, 0.9670, 0.9605, 0.9443, 0.9651, 0.9863, 0.9506,
         0.9498, 0.9978, 0.9403, 0.9889, 0.9631, 0.9752, 0.9745],
        [0.9663, 0.9825, 0.9482, 0.9544, 0.9788, 0.9620, 0.9557, 1.0020, 0.9760,
         0.9847, 0.9515, 0.9465, 0.9569, 0.9694, 0.9821, 0.9788],
        [0.9717, 0.9845, 0.9337, 0.9739, 0.9665, 0.9817, 0.9783, 0.9816, 0.9443,
         0.9764, 0.9740, 0.9756, 0.9467, 0.9624, 0.9604, 0.9220],
        [0.9741, 0.9794, 0.9765, 0.9688, 0.9724, 0.9737, 0.9858, 0.9640, 0.9680,
         0.9712, 0.9841, 0.9474, 1.0056, 0.9641, 0.9403, 0.9414],
        [0.9497, 0.9469, 0.9456, 0.9620, 0.9467, 0.9635, 0.9703, 0.9685, 0.9467,
         0.9675, 0.9786, 0.9339, 0.9441, 0.9458, 0.9739, 0.9822],
        [0.9544, 0.9802,

zeros
----------------------------------------------------------------------------------------------------
Previous layer shape is  torch.Size([16, 96, 3, 3])
shape of layer: model 0 torch.Size([8, 16, 9])
shape of layer: model 1 torch.Size([8, 16, 9])
shape of previous transport map torch.Size([16, 16])
current, current
T_var_conv: torch.Size([9, 16, 16])
fc_layer0_weight_data: torch.Size([9, 8, 16])
Processing the coordinates to form ground_metric
dont leave off the squaring of the ground metric
dist: tensor([[0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794, 0.9742, 0.9829, 0.9726],
        [0.9691, 1.0053, 0.9737, 0.9510, 0.9794

returns a uniform measure of cardinality:  8
returns a uniform measure of cardinality:  8
the transport map is  tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')
Ratio of trace to the matrix sum:  tensor(nan, device='cuda:0')
Here, trace is 0.0 and matrix sum is 0.0 
col sum: tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'), row sum: tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
this is past correction for weight mode
Shape of aligned wt is  torch.Size([8, 32, 9])
Shape of fc_layer0_weight_data is  torch.Size([8, 32, 9])
model1 before: tensor([[[-3.9802e-02, -9.0351e-02,  3.6045e-02,  ...,  2.7907e-02,
           1.5760e-02,  2.1942e-02],
         [ 3.0176e-04,  4.05

model1 after: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.,

In [6]:
# model_fused

## Load Data

In [7]:
train_ds = load_data(args["ds_name"], "train", vendor=args["vendor"])
val_ds = load_data(args["ds_name"], "val", vendor=args["vendor"])

Loading dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 104/104 [00:00<00:00, 134.70it/s]


In [8]:
train_indices = torch.arange(args["num_retrain_samples"])
train_ds_subset = Subset(train_ds, train_indices)

In [9]:
len(train_ds_subset), len(train_ds)

(5, 1740)

In [10]:
len(val_ds)

104

In [11]:
ds_dict = {
    "train": train_ds_subset,
    "val": val_ds
}

## Models

In [32]:
model = model_fused
# model = model1
# model = model2
lit_model = TrainSeg(ds_dict, model.to(torch.device("cpu")))

In [33]:
logger, time_stamp = get_logger(args["save_dir"], args["log_name"])
callbacks = [
    ValVisualizationSeg(save_interval=1),
    ModelCheckpoint(os.path.join(args["save_dir"], args["log_name"], time_stamp, "checkpoints/"),
                    monitor="val_dsc", mode="max")
]

In [34]:
# trainer = pl.Trainer(
#     accelerator="gpu",
#     devices=1,
#     fast_dev_run=2
# )

In [35]:
# trainer.fit(lit_model)

In [36]:
trainer = pl.Trainer(
    accelerator=args["accelerator"],
    devices=1,
    logger=logger,
    callbacks=callbacks,
    num_sanity_val_steps=-1,
    # precision=16,
#     max_epochs=args["num_retrain_epochs"]
    max_epochs=2
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [37]:
trainer.fit(lit_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | model   | UNet       | 41.5 K
1 | loss_fn | DiceCELoss | 0     
---------------------------------------
41.5 K    Trainable params
0         Non-trainable params
41.5 K    Total params
0.166     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [38]:
trainer.validate(lit_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_dsc            0.43588897585868835
        val_loss            0.39802324771881104
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.39802324771881104, 'val_dsc': 0.43588897585868835}]